In [1]:
import requests
import pandas as pd
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import openpyxl
import os
from dotenv import load_dotenv
from datetime import datetime
import json



In [2]:
# Load the environment variables
load_dotenv()

# API key
map_api_key = os.getenv('GOOGLE_MAPS_API_KEY')
sheet_api_key = os.getenv('GOOGLE_SHEET_API_KEY')


In [3]:
sheet_url = "https://docs.google.com/spreadsheets/d/1mAXInBMDYVKjZ5DVz5OlwDx8co33BuOF-49Vyog57NE/edit?gid=0#gid=0"

# Test sheet accessibility
sheet_id = sheet_url.split("/d/")[1].split("/edit")[0]
test_url = f"https://sheets.googleapis.com/v4/spreadsheets/{sheet_id}?key={sheet_api_key}"
response = requests.get(test_url)
print("Response status:", response.status_code)
print("Response:", response.text[:200])  # Print first 200 characters of response

Response status: 200
Response: {
  "spreadsheetId": "1mAXInBMDYVKjZ5DVz5OlwDx8co33BuOF-49Vyog57NE",
  "properties": {
    "title": "Master NETWORK OF NETWORKS MAP SHEET",
    "locale": "en_US",
    "autoRecalc": "ON_CHANGE",
    "t


In [4]:
def test_sheet_access(sheet_url, sheet_api_key):
    try:
        # Extract sheet ID from URL
        sheet_id = sheet_url.split("/d/")[1].split("/edit")[0]
        
        # Test 1: Basic API Access
        print("Test 1: Testing basic API access...")
        test_url = f"https://sheets.googleapis.com/v4/spreadsheets/{sheet_id}?key={sheet_api_key}"
        response = requests.get(test_url)
        print(f"API Response Status: {response.status_code}")
        
        if response.status_code != 200:
            print("❌ Failed to access the sheet via API")
            return False
            
        # Test 2: Get Sheet Metadata
        print("\nTest 2: Retrieving sheet metadata...")
        sheet_data = response.json()
        print(f"Sheet Title: {sheet_data['properties']['title']}")
        print(f"Sheet ID: {sheet_data['spreadsheetId']}")
        
        # Test 3: Try to read actual data with a smaller range
        print("\nTest 3: Attempting to read sheet data...")
        values_url = f"https://sheets.googleapis.com/v4/spreadsheets/{sheet_id}/values/A1:A1?key={sheet_api_key}"
        values_response = requests.get(values_url)
        
        if values_response.status_code == 200:
            values_data = values_response.json()
            if 'values' in values_data:
                print("\nFirst cell of data:")
                print(values_data['values'])
                print("\n✅ Successfully read sheet data!")
            else:
                print("❌ No data found in the sheet")
        else:
            print(f"❌ Failed to read sheet data. Status code: {values_response.status_code}")
            print(f"Error details: {values_response.text}")
            
        return True
        
    except Exception as e:
        print(f"❌ Error occurred: {str(e)}")
        return False

# Execute the test
print("Starting sheet accessibility test...")
sheet_url = "https://docs.google.com/spreadsheets/d/1mAXInBMDYVKjZ5DVz5OlwDx8co33BuOF-49Vyog57NE/edit?gid=0#gid=0"
sheet_api_key = os.getenv('GOOGLE_SHEET_API_KEY')
test_sheet_access(sheet_url, sheet_api_key)

Starting sheet accessibility test...
Test 1: Testing basic API access...
API Response Status: 200

Test 2: Retrieving sheet metadata...
Sheet Title: Master NETWORK OF NETWORKS MAP SHEET
Sheet ID: 1mAXInBMDYVKjZ5DVz5OlwDx8co33BuOF-49Vyog57NE

Test 3: Attempting to read sheet data...

First cell of data:
[['Affiliated Project']]

✅ Successfully read sheet data!


True

In [5]:
def inspect_sheet_structure(sheet_url, sheet_api_key):
    """Inspect the structure of the Google Sheet."""
    try:
        # Extract sheet ID from URL
        sheet_id = sheet_url.split("/d/")[1].split("/edit")[0]
        
        # Create service
        service = build('sheets', 'v4', developerKey=sheet_api_key)
        
        # Get the first sheet's data
        sheet = service.spreadsheets()
        result = sheet.values().get(
            spreadsheetId=sheet_id,
            range='A:Z'  # Get all columns
        ).execute()
        
        values = result.get('values', [])
        
        if values:
            print("\nHeaders:")
            print(values[0])
            
            if len(values) > 1:
                print("\nFirst row of data:")
                print(values[1])
        else:
            print("No data found in sheet")
            
    except Exception as e:
        print(f"Error inspecting sheet: {str(e)}")
        raise

# Execute the inspection
print("Starting sheet structure inspection...")
sheet_url = "https://docs.google.com/spreadsheets/d/1mAXInBMDYVKjZ5DVz5OlwDx8co33BuOF-49Vyog57NE/edit?gid=0#gid=0"
sheet_api_key = os.getenv('GOOGLE_SHEET_API_KEY')
inspect_sheet_structure(sheet_url, sheet_api_key)

Starting sheet structure inspection...

Headers:
['Affiliated Project', 'Title', 'Map Differentiation', 'Activities Occurring', 'Address']

First row of data:
['Canadian BioGenome Project (CANBP)', 'BC Cancer', 'Headquarters', 'DNA Sequencing (long reads), DNA Sequencing (short reads), Bioinformatics (assembly), Bioinformatics (annotation)', '675 W 10th Ave, Vancouver, BC V5Z 0B4, Canada']


In [6]:
# fetch data from publicly accessible google sheet
def get_coordinates(address, map_api_key):
    """Fetch coordinates for a given address using Google Maps Geocoding API."""
    if not address:
        print("Address is empty or invalid.")
        return None, None

    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address.replace(' ', '+')}&key={map_api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            location = data['results'][0]['geometry']['location']
            return location['lat'], location['lng']
        else:
            print(f"Failed to geocode address: {address} | Status: {data['status']}")
            return None, None
    else:
        print(f"API request error: {response.status_code}")
        return None, None

def get_sheet_metadata(sheet_id, sheet_api_key):
    """Retrieve metadata of all sheets in the Google Sheets file."""
    try:
        # Create credentials object with just the API key
        from google.oauth2.credentials import Credentials
        from google.auth.credentials import AnonymousCredentials
        
        # Build the service with anonymous credentials and API key
        service = build(
            'sheets', 
            'v4',
            credentials=AnonymousCredentials(),
            developerKey=sheet_api_key
        )
        
        # Get the spreadsheet metadata
        sheet_metadata = service.spreadsheets().get(spreadsheetId=sheet_id).execute()
        sheets = sheet_metadata.get('sheets', [])
        return {sheet['properties']['sheetId']: sheet['properties']['title'] for sheet in sheets}
    except HttpError as err:
        print(f"Error details: {err}")
        if "403" in str(err):
            print("\nThis might mean either:")
            print("1. The Google Sheet is not publicly accessible")
            print("2. The API key doesn't have access to Google Sheets API")
            print("\nPlease check:")
            print("1. Share settings of your Google Sheet (make it 'Anyone with the link can view')")
            print("2. Enable Google Sheets API in Google Cloud Console")
        raise Exception(f"Error fetching sheet metadata: {err}")


In [8]:
def process_google_sheet(sheet_url, map_api_key, sheet_api_key, output_format='excel'):
    """
    Process all sheets in a Google Sheets file, add latitude and longitude.
    Args:
        sheet_url: URL of the Google Sheet
        map_api_key: Google Maps API key
        sheet_api_key: Google Sheets API key
        output_format: 'excel' or 'json' (default: 'excel')
    """
    try:
        # Validate output format
        if output_format not in ['excel', 'json']:
            raise ValueError("output_format must be either 'excel' or 'json'")

        # Retrieve the Google Sheet ID from the URL
        sheet_id = sheet_url.split("/d/")[1].split("/edit")[0]
        
        # Create timestamp for output file
        timestamp = datetime.now().strftime("%Y%m%d")
        
        # Initialize data structures based on output format
        all_json_data = [] if output_format == 'json' else None
        writer = pd.ExcelWriter(f"affiliate_network_list_{timestamp}.xlsx", engine='openpyxl') if output_format == 'excel' else None

        # Initialize counters for summary
        total_records = 0
        sheet_records = {}
        processed_sheets = 0

        # Base export URL for multiple sheets
        base_csv_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid="

        # Get sheet metadata and process sheets
        sheet_metadata = get_sheet_metadata(sheet_id, sheet_api_key)
        total_sheets = len(sheet_metadata)
        print(f"\nTotal number of sheets found: {total_sheets}")

        for gid, sheet_name in sheet_metadata.items():
            print(f"\nProcessing sheet: {sheet_name}")
            
            # Generate the CSV URL for the current sheet
            csv_url = f"{base_csv_url}{gid}"

            # Read the current sheet into a DataFrame
            data = pd.read_csv(csv_url)
            
            if 'Address' not in data.columns:
                print(f"Warning: No 'Address' column found in sheet '{sheet_name}'. Skipping...")
                continue

            # Add latitude and longitude columns
            data['Latitude'] = None
            data['Longitude'] = None

            # Process each row
            total_rows = len(data)
            print(f"Processing {total_rows} rows...")

            # Process all rows
            rows_to_process = data

            for index, row in rows_to_process.iterrows():
                address = row['Address']
                if pd.isna(address):
                    print(f"Skipping row {index + 1}: No address found")
                    continue
                   
                print(f"\nProcessing row {index + 1}/{total_rows}")
                print(f"Address: {address}")
                
                latitude, longitude = get_coordinates(address, map_api_key)
                data.at[index, 'Latitude'] = latitude
                data.at[index, 'Longitude'] = longitude
                
                print(f"Coordinates: {latitude}, {longitude}")

                # Create JSON entry if json format is selected
                if output_format == 'json':
                    json_entry = {
                        "projectName": row.get('Affiliated Project', ''),
                        "institution": row.get('Title', ''),
                        "type": row.get('Map Differentiation', ''),
                        "activities": row.get('Activities Occurring', ''),
                        "address": row.get('Address', ''),
                        "latitude": latitude,
                        "longitude": longitude
                    }
                    all_json_data.append(json_entry)

            # Write to Excel if excel format is selected
            if output_format == 'excel':
                data.to_excel(writer, sheet_name=sheet_name, index=False)
                print(f"✅ Processed sheet '{sheet_name}' and added it to the Excel workbook.")
            
            # Update counters
            sheet_records[sheet_name] = len(data)
            total_records += len(data)
            processed_sheets += 1

        # Save final output
        if output_format == 'excel':
            writer.close()
            print(f"\n✅ Excel file saved as 'affiliate_network_list_{timestamp}.xlsx'")
        else:  # json format
            output_json_file = f"affiliate_network_list_{timestamp}.json"
            with open(output_json_file, 'w', encoding='utf-8') as f:
                json.dump(all_json_data, f, ensure_ascii=False, indent=2)
            print(f"✅ JSON file saved as '{output_json_file}'")

        # Print summary statistics
        print("\n=== Summary Statistics ===")
        print(f"Total number of sheets: {total_sheets}")
        print(f"Number of processed sheets: {processed_sheets}")
        print(f"\nOutput format: {output_format}")
        print("\nRecords per sheet:")
        for sheet_name, count in sheet_records.items():
            print(f"- {sheet_name}: {count} records")
        print(f"\nTotal number of records across all sheets: {total_records}")

    except Exception as e:
        print(f"Error processing sheet: {str(e)}")
        raise

# Example usage:
print("Starting sheet processing...")
process_google_sheet(sheet_url, map_api_key, sheet_api_key, output_format='excel')  # or 'json'

Starting sheet processing...

Total number of sheets found: 36

Processing sheet: Canadian BioGenome Project
Processing 5 rows...

Processing row 1/5
Address: 675 W 10th Ave, Vancouver, BC V5Z 0B4, Canada
Coordinates: 49.2626787, -123.1194676

Processing row 2/5
Address: 170 Elizabeth St, Toronto, ON M5G 1E8, Canada
Coordinates: 43.6576007, -79.38650109999999

Processing row 3/5
Address: 845 Rue Sherbrooke O, Montréal, QC H3A 0G4, Canada
Coordinates: 45.5063178, -73.5767035

Processing row 4/5
Address: CCIS 3-170. University of Alberta Edmonton, AB, Canada T6G 2E9
Coordinates: 53.5281589, -113.5257417

Processing row 5/5
Address: 570 W 7th Ave, Vancouver, BC V5Z 4S6, Canada
Coordinates: 49.2647669, -123.1168384
✅ Processed sheet 'Canadian BioGenome Project' and added it to the Excel workbook.

Processing sheet: CanSeq150
Processing 5 rows...
Skipping row 1: No address found

Processing row 2/5
Address: 675 W 10th Ave, Vancouver, BC V5Z 0B4, Canada
Coordinates: 49.2626787, -123.1194676


c:\Users\fchen13\AppData\Local\miniconda3\envs\myenvironment\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Processing 5 rows...

Processing row 1/5
Address: 127 West Youyi Road, Beilin District, Xi'an, Shaanxi, China, 710072
Coordinates: 34.24149999999999, 108.91147

Processing row 2/5
Address: Nørregade 10, 1172 København, Denmark
Coordinates: 55.6801421, 12.5714877

Processing row 3/5
Address: 3 Taicheng Rd, Yangling District, Xianyang, Shaanxi, China, 712100
Coordinates: 34.26166, 108.07534

Processing row 4/5
Address: 120 Xianlie Middle Rd, Yuexiu District, Guangzhou, Guangdong, China, 510070
Coordinates: 23.14379, 113.30335

Processing row 5/5
Address: No.17 Longxin Road, Kunming, Yunnan, 650201, P.R.China
Coordinates: 24.90353, 102.79028
✅ Processed sheet 'Ungulates Project' and added it to the Excel workbook.

Processing sheet: Threatened Species Initiative (TSI)
Processing 6 rows...

Processing row 1/6
Address: RMC Gunn Building B19, The University of Sydney, Sydney, 2006 NSW Australia
Coordinates: -33.8866012, 151.1839733

Processing row 2/6
Address: 7 Dick Perry Avenue KENSINGTON 

c:\Users\fchen13\AppData\Local\miniconda3\envs\myenvironment\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Processing 10 rows...

Processing row 1/10
Address: Georg-Voigt-Str. 14-16, 60388 Frankfurt am Main, Germany
Coordinates: 50.1155497, 8.6502796

Processing row 2/10
Address: Am Museum 1 02826 Görlitz Germany
Coordinates: 51.1534602, 14.9865865

Processing row 3/10
Address: 57 rue Cuvier, 75005 Paris, France
Coordinates: 48.8438705, 2.3567311

Processing row 4/10
Address: Adenauerallee 127, 53113 Bonn, Germany
Coordinates: 50.7233807, 7.113946599999999

Processing row 5/10
Address: Burgring 7, 1010 Wien, Austria
Coordinates: 48.205213, 16.3598442

Processing row 6/10
Address: Hinxton, Cambridgeshire,, CB10 1SA. UK
Coordinates: 52.07976379999999, 0.1853909

Processing row 7/10
Address: Zülpicher Straße 47b, 50674 Köln, Germany
Coordinates: 50.92670510000001, 6.9357053

Processing row 8/10
Address: 1 Rue Eugène Boeckel, 67000 Strasbourg, France
Coordinates: 48.575845, 7.739033999999999

Processing row 9/10
Address: Untere Karspüle 2, 37073 Goettingen, Germany
Coordinates: 51.5373968, 9.93

...

Processing 15 rows...

Processing row 1/15
Address: BGI Research, Shenzhen 518083, China
Coordinates: 22.5989799, 114.31321

Processing row 2/15
Address: BGI Research, Wuhan 430074, China
Coordinates: 30.5311927, 114.3400858

Processing row 3/15
Address: China National GeneBank, Jinsha Road, Dapeng New District, Shenzhen, Guangdong Province, China
Coordinates: 22.7510612, 114.3966867

Processing row 4/15
Address: Department of Biological Sciences and Department of Medicine, University of Alberta, Edmonton, Alberta, Canada
Coordinates: 53.5290289, -113.5255352

Processing row 5/15
Address: Key Laboratory of Southern Subtropical Plant Diversity, Fairy Lake Botanical Garden, Shenzhen & Chinese Academy of Sciences, Shenzhen, China
Coordinates: 22.5775992, 114.1825737

Processing row 6/15
Address: African Orphan Crops Consortium,World Agroforestry Centre (ICRAF), Nairobi 00100, Kenya
Coordinates: -1.236467, 36.819017

Processing row 7/15
Address: Department of Plant Microbe Interactions, Ma

c:\Users\fchen13\AppData\Local\miniconda3\envs\myenvironment\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Processing 6 rows...

Processing row 1/6
Address: School of Ecology and Environment, Northwestern Polytechnical University, Xi'an 710072, China
Coordinates: 34.2459632, 108.9138675

Processing row 2/6
Address: CAS and Shandong Province Key Laboratory of Experimental Marine Biology, Center for Ocean Mega-Science, Institute of Oceanology, Chinese Academy of Sciences, Qingdao, China
Coordinates: 36.0662299, 120.38299

Processing row 3/6
Address: Institute of Deep-sea Science and Engineering, Chinese Academy of Sciences, Sanya 572000, China
Coordinates: 18.2524799, 109.51209

Processing row 4/6
Address: BGI-Qingdao, BGI-Shenzhen, Qingdao, China
Coordinates: 36.0662299, 120.38299

Processing row 5/6
Address: Biomarker Technologies Co，LTD，Beijing, China
Coordinates: 39.904211, 116.407395

Processing row 6/6
Address: Xi’an Haorui Genomics Technology Co., LTD, Xi’an, China
Coordinates: 34.2658138, 108.9540936
✅ Processed sheet 'Cartilaginous Fish Genome Project' and added it to the Excel workb

c:\Users\fchen13\AppData\Local\miniconda3\envs\myenvironment\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Processing 10 rows...

Processing row 1/10
Address: Departament de Genètica, Microbiologia i Estadística. Avinguda Diagonal, 643. 08028 Barcelona (Spain)
Coordinates: 41.3855347, 2.1198039

Processing row 2/10
Address: Pg. Marítim de la Barceloneta, 37, Ciutat Vella, 08003 Barcelona (Spain)
Coordinates: 41.3854833, 2.1958498

Processing row 3/10
Address: Carrer de Baldiri Reixac, 4, torre i, Distrito de Les Corts, 08028 Barcelona (Spain)
Coordinates: 41.381849, 2.1165617

Processing row 4/10
Address: P° Picasso s/n, Parc Ciutadella, 08003, Barcelona, Spain
Coordinates: 41.388123, 2.1860152

Processing row 5/10
Address: Passeig del Migdia, s/n, Sants-Montjuïc, 08038 Barcelona (Spain)
Coordinates: 41.3615322, 2.1603489

Processing row 6/10
Address: Prat de la Creu, 68-76, AD500 Andorra la Vella (Andorra)
Coordinates: 42.5062207, 1.5213598

Processing row 7/10
Address: Carrer del Carme, 47 08001 Barcelona.
Coordinates: 41.3813121, 2.1692108

Processing row 8/10
Address: Carrer Blaise Pasc